**Buffering Water Flowlines**

Water data has been cleaned by another notebook, so we can focus on buffering. First, let's set up the project path and import helpers.

In [2]:
%run ../../bootstrap.py
setup_project_path()

from scripts.io_helpers import read_interim_layer, export_interim
from scripts.data_helpers import to_buffer_crs
from scripts import data_config as dc

Next, let's load the data and project to the buffering crs.

In [3]:
# Load cleaned MVUM data
gdf = read_interim_layer("flowline_clean")

# Ensure correct CRS
gdf_projected = to_buffer_crs(gdf)

Finally, let's buffer and export.

In [ ]:
# Buffer
gdf_buffered = gdf_projected.copy()
gdf_buffered["geometry"] = gdf_buffered.geometry.buffer(dc.BUFFER_SETTINGS_METERS["water"])

# Export
export_interim(gdf_buffered, "water_flowline_buffered", driver="GPKG", verbose=True)